# 07 — Time Series (Univariate) — PyCaret 3
**Dataset:** Daily-Min-Temperatures  
**Goal:** Forecast 30 days.

In [8]:
# Install latest stable PyCaret 3.x and common GPU-capable libs
# !pip -q install --upgrade pycaret xgboost catboost lightgbm plotly gradio

import sys, sklearn, pycaret, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("scikit-learn:", sklearn.__version__)
print("pycaret:", pycaret.__version__)

# Check GPU
try:
    import subprocess, shutil
    if shutil.which("nvidia-smi"):
        print("\nGPU detected:")
        print(subprocess.check_output(["nvidia-smi"], text=True))
    else:
        print("\nNo NVIDIA GPU detected in this runtime. (Colab: Runtime -> Change runtime type -> GPU)")
except Exception as e:
    print("GPU check error:", e)

Python: 3.11.11 (main, Feb  5 2025, 19:11:07) [Clang 19.1.6 ]
Platform: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
scikit-learn: 1.4.2
pycaret: 3.3.2

GPU detected:
Sun Nov  2 16:42:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.05              Driver Version: 580.88         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   35C    P8             10W /  200W |     9

In [5]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv")
df['Date'] = pd.to_datetime(df['Date'])
# Set the frequency to daily when creating the time series
y = df.set_index('Date')['Temp'].asfreq('D')

In [3]:
print("Series length:", len(y), "from", y.index.min(), "to", y.index.max())
display(y.head())

Series length: 3650 from 1981-01-01 00:00:00 to 1990-12-31 00:00:00


Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Name: Temp, dtype: float64

In [10]:
from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()
exp.setup(
    data=y, 
    fh=30, 
    fold=2,
    session_id=42, 
    seasonal_period=7, 
    n_jobs=1,
    verbose=False,
    numeric_imputation_target='mean',
    use_gpu=True  # Enable GPU usage
)
# Compare models that can leverage GPU
best = exp.compare_models()
final = exp.finalize_model(best)
preds = exp.predict_model(final)
fig = exp.plot_model(final, plot='forecast')

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Th

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.7257,0.7168,2.1437,2.6886,0.1850,0.1811,0.0184,0.3650
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.7257,0.7168,2.1437,2.6886,0.1850,0.1811,0.0184,0.3350
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.7281,0.7197,2.1509,2.6993,0.1869,0.1816,0.0110,0.3750
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.7345,0.7303,2.1697,2.7392,0.1908,0.1828,-0.0198,0.4300
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.7443,0.7386,2.1987,2.7703,0.1926,0.1853,-0.0424,0.3750
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.7445,0.7388,2.1992,2.7710,0.1927,0.1853,-0.0430,0.3250
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.7445,0.7388,2.1992,2.7710,0.1927,0.1853,-0.0430,0.3400
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,0.7559,0.7474,2.2330,2.8034,0.1896,0.1886,-0.0752,0.3350
auto_arima,Auto ARIMA,0.7645,0.7620,2.2585,2.8581,0.1842,0.1909,-0.1134,88.5950
croston,Croston,0.7726,0.7953,2.2821,2.9831,0.1922,0.1953,-0.2054,0.0250
